In [15]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/CS5780/Detect-AI-Generated-Text"

# Verify the current working directory
print("Current working directory:", os.getcwd())

# List the contents of the folder
folder_contents = os.listdir()
print(folder_contents)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS5780/Detect-AI-Generated-Text
Current working directory: /content/drive/MyDrive/CS5780/Detect-AI-Generated-Text
['main.ipynb', 'mytrain.csv']


In [16]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification

import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv('mytrain.csv')
df = df.iloc[:, -2:]
df

,text,generated
0,Candidate what do you think of when I say that...,0
1,"Ever since it was first invented, the automobi...",0
2,Have you ever come back to school after a long...,0
3,"Imagine you receiving advice from someone, but...",0
4,If you waych the news chaces are you probelly ...,0
...,...,...
395,Title: The Pros and Cons of a Four-Day Work We...,1
396,\nWhether summer projects for students should ...,1
397,Building self-esteem by accomplishing goals ca...,1
398,Title: Minister Winston's Selfless Acts of Kin...,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['generated'], test_size=0.2, random_state=42)
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [19]:
# Assuming X_train, X_test, y_train, y_test are your training and testing data
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize and preprocess the data
def preprocess_data(texts, labels):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs["input_ids"].squeeze()
    attention_mask = inputs["attention_mask"].squeeze()
    return input_ids, attention_mask, labels

# Preprocess the training and testing data
train_dataset = TensorDataset(*[torch.tensor(t) for t in preprocess_data(*zip(*train_data))])
test_dataset = TensorDataset(*[torch.tensor(t) for t in preprocess_data(*zip(*test_data))])

# Compile and fit the model
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

epochs = 2
for epoch in range(epochs):
    model.train()
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}, Training", unit="batch", dynamic_ncols=True)

    for input_ids, attention_mask, labels in train_iterator:
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({"Loss": loss.item()})

    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    test_iterator = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs}, Validation", unit="batch", dynamic_ncols=True)

    with torch.no_grad():
        for input_ids, attention_mask, labels in test_iterator:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).item()  # Use item() for scalar tensor
            val_labels.append(labels.item())  # Use item() for scalar tensor
            val_preds.append(preds)

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-0235c2c76095>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

Epoch 1/2, Validation Accuracy: 0.5000


Epoch 2/2, Validation: 100%|██████████| 80/80 [00:33<00:00,  2.42batch/s]

Epoch 2/2, Validation Accuracy: 0.9500
